A Prediction that totally based on seed difference: 
[kaggle 1](https://www.kaggle.com/ateplyuk/lgbm-str-w)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

In [3]:
t_res = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneyCompactResults.csv')
t_ds = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneySeeds.csv')
sub = pd.read_csv('womens-machine-learning-competition-2019/WSampleSubmissionStage1.csv')

In [4]:
print(t_res.shape)
t_res.head()

(1323, 8)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [5]:
t_ds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [6]:
sub.head()

,ID,Pred
0,2014_3103_3107,0.5
1,2014_3103_3113,0.5
2,2014_3103_3119,0.5
3,2014_3103_3124,0.5
4,2014_3103_3140,0.5


In [7]:
# seed to int
t_ds['seed_int'] = t_ds.Seed.apply(lambda a : int(a[1:3]))

In [8]:
drop_lbls = ['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT']
t_ds.drop(labels=['Seed'], inplace=True, axis=1)
t_res.drop(labels=drop_lbls, inplace=True, axis=1)

t_res.head()

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208


In [9]:
ren1 = {'TeamID':'WTeamID', 'seed_int':'WS'}
ren2 = {'TeamID':'LTeamID', 'seed_int':'LS'}

df1 = pd.merge(left=t_res, right=t_ds.rename(columns=ren1), how='left', on=['Season', 'WTeamID'])
df2 = pd.merge(left=df1, right=t_ds.rename(columns=ren2), on=['Season', 'LTeamID'])

df_w = pd.DataFrame()
df_w['dff'] = df2.WS - df2.LS  # seed diff
df_w['rsl'] = 1

df_l = pd.DataFrame()
df_l['dff'] = -df_w['dff']
df_l['rsl'] = 0

df_prd = pd.concat((df_w, df_l))

df_prd.head()

,dff,rsl
0,-13,1
1,-11,1
2,-13,1
3,-3,1
4,3,1


In [10]:
df2.head()

,Season,WTeamID,LTeamID,WS,LS
0,1998,3104,3422,2,15
1,1998,3112,3365,3,14
2,1998,3163,3193,2,15
3,1998,3198,3266,7,10
4,1998,3203,3208,10,7


In [11]:
X = df_prd.dff.values.reshape(-1,1)
y = df_prd.rsl.values

In [12]:
X_test = np.zeros(shape=(len(sub), 1))

In [14]:
# get only seed diff
for ind, row in sub.iterrows():
    yr, o, t = [int(x) for x in row.ID.split('_')]  
    X_test[ind, 0] = t_ds[(t_ds.TeamID == o) & (t_ds.Season == yr)].seed_int.values[0] - t_ds[(t_ds.TeamID == t) & (t_ds.Season == yr)].seed_int.values[0]

In [16]:
X_test.shape

(10080, 1)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

In [19]:
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.01,
          "num_leaves": 31,
          "min_data_in_leaf": 10,
          "min_child_samples": 10,
          "metric": "binary_logloss"
          }

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_val, y_val, reference=dtrain)

bst = lgb.train(params, dtrain, 1000, valid_sets=dvalid, verbose_eval=200, early_stopping_rounds=50)

Train until valid scores didn't improve in 50 rounds.
[200]	valid_0's binary_logloss: 0.438127
[400]	valid_0's binary_logloss: 0.417324
[600]	valid_0's binary_logloss: 0.414788
[800]	valid_0's binary_logloss: 0.414452
[1000]	valid_0's binary_logloss: 0.414407


In [21]:
test_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

test_pred[:10]

array([7.69219126e-01, 2.40751952e-01, 5.00000000e-01, 2.26211837e-05,
       4.57015430e-01, 1.52557423e-01, 2.30780874e-01, 2.26211837e-05,
       6.91420302e-01, 1.52557423e-01])